In [2]:
import json
import requests # to download some resources
import os # file operations
import numpy as np # linear algebra
import pandas as pd # data processing

In [3]:
# Consolidating answers from the whole sample
# Folder Path
folder_json = f"../data/4-summary-responses-json"

# Lista para almacenar los DataFrames
dfs = []

# Recorrer todos los archivos en la carpeta
for file_name in os.listdir(folder_json):
    if file_name.endswith(".json"):  # Filtrar solo JSONs
        file_path = os.path.join(folder_json, file_name)  # Ruta completa del archivo
        
        # Cargar el JSON y convertirlo en DataFrame
        df = pd.read_json(file_path)  # Aquí estaba el error
        dfs.append(df)


# Concatenar todos los DataFrames en uno solo
combined_data = pd.concat(dfs, ignore_index=True)

# Collapse by country and question
combined_data = combined_data.groupby(['country', 'question'], as_index=False).agg({
    'answer': lambda x: ', '.join(x.dropna().astype(str).tolist()),
    'supporting_chunks': 'first',
    'answer_full': 'first',
    'supporting_text': 'first'
})

combined_data['question'] = combined_data['question'].astype(str)
combined_data['country'] = combined_data['country'].astype(str)
combined_data = combined_data.rename(columns={"answer": "answer_chunking"})
combined_data = combined_data[~combined_data['question'].isin(['period_start', 'period_end'])]

# Load the annotations
annotations_df = pd.read_excel('../data/5-annotations/annotations.xlsx')
annotations_df['question'] = annotations_df['question'].astype(str)
annotations_df['answer'] = annotations_df['answer'].astype(str)

# Joining with annotations
combined_data = combined_data.merge(annotations_df, how='inner', on=['country', 'question'], suffixes=('', '_right'))

output_path = f"../data/6-performance/combined_data.xlsx"  # Nombre del archivo Excel
combined_data.to_excel(output_path, index=False)

print(f"Archivo Excel guardado en: {output_path}")


Archivo Excel guardado en: ../data/6-performance/combined_data.xlsx


In [11]:
combined_data.head()

,country,question,answer_chunking,supporting_chunks,answer_full,supporting_text,answer,comment_input_for_protocol,supporting_text_when_true_answer_is_yes_but_prediction_is_no,chunking_binary,full_binary,annotation_binary
0,chile,1,yes,[Generar instancias de capacitación locales pa...,yes,The NAP mentions specific actions in the human...,yes,NaN,NaN,1,1,1
1,chile,2,yes,[111.6. Prevenir y controlar infecciones en la...,yes,The NAP includes actions to prevent and contro...,yes,NaN,NaN,1,1,1
2,chile,3,yes,[111.5. Incorporar medidas regulatorias en rel...,yes,The NAP emphasizes the importance of environme...,yes,NaN,NaN,1,1,1
3,chile,4,yes,[11.6. Fortalecer la vigilancia de la resisten...,yes,The NAP addresses the regulation of antimicrob...,yes,NaN,NaN,1,1,1
4,chile,5,yes,[se busca integrar los hallazgos de la vigilan...,yes,The NAP outlines the need for annual or biannu...,yes,"The LLM prediction is *correct*, but I think t...",NaN,1,1,1
